### Feature Engineering
- At this stage we add calculated fields to the original fundamentals.

In [14]:
import pandas as pd
file = "/Users/imperator/Documents/STUDIES/UNIVERSITY OF GHANA/RESEARCH WORK/CORCHIL KELLY KWAME/PORTFOLIO OPTIMIZATION/PROJECT CODE/Dimensionality-Reduction-PortfolioOptimization/ETFs_datasets/Final_csv/merged_etfs.csv"
df = pd.read_csv(file)
df.head()

,Ticker,Date,Close,High,Low,Open,Volume
0,SPY,2005-04-11,81.121849,81.348544,80.943246,81.259242,44945000.0
1,SPY,2005-04-12,81.540894,81.788196,80.421168,80.984466,86144800.0
2,SPY,2005-04-13,80.579117,81.609540,80.462332,81.444669,65949000.0
3,SPY,2005-04-14,79.528145,80.716570,79.528145,80.647876,96119800.0
4,SPY,2005-04-15,78.415268,79.823511,78.380918,79.507515,128677300.0


In [17]:
print(df.columns)

Index(['Ticker', 'Date', 'Close', 'High', 'Low', 'Open', 'Volume'], dtype='object')


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54691 entries, 0 to 54690
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Ticker  54691 non-null  object 
 1   Date    54691 non-null  object 
 2   Close   54680 non-null  float64
 3   High    54680 non-null  float64
 4   Low     54680 non-null  float64
 5   Open    54680 non-null  float64
 6   Volume  54680 non-null  float64
dtypes: float64(5), object(2)
memory usage: 2.9+ MB


In [35]:
import pandas as pd
import numpy as np
from ta.trend import MACD, SMAIndicator, EMAIndicator, ADXIndicator
from ta.volatility import AverageTrueRange, BollingerBands
from ta.momentum import RSIIndicator
from technical_indicator.volume import OnBalanceVolume
import os


def calculate_indicators_with_moving_averages(file_path):
    # Load data
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found at {file_path}")

    df = pd.read_csv(file_path)

    # Validate required columns
    required_cols = ['Close', 'High', 'Low', 'Open', 'Volume']
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Missing columns: {', '.join(missing_cols)}")

    df.sort_values(by=['Ticker', 'Date'] if 'Date' in df.columns else ['Ticker'], inplace=True)
    df['returns'] = df.groupby('Ticker')['Close'].transform(
    lambda x: x.pct_change() if len(x) >= 252 else np.nan
)

    # Trend Indicators
    df['SMA_20'] = df.groupby('Ticker')['Close'].transform(lambda x: SMAIndicator(close=x, window=20).sma_indicator())
    df['EMA_20'] = df.groupby('Ticker')['Close'].transform(lambda x: EMAIndicator(close=x, window=20).ema_indicator())

    bb = df.groupby('Ticker', group_keys=False).apply(
    lambda x: BollingerBands(close=x['Close'], window=20).bollinger_hband()
)
    df['Bollinger_High'] = bb.values

    # Momentum Indicators
    macd = df.groupby('Ticker', group_keys=False).apply(
    lambda x: MACD(close=x['Close'], window_slow=26, window_fast=12, window_sign=9).macd()
)
    df['MACD'] = macd.values


    # Volatility
    df['ATR_14'] = df.groupby('Ticker').apply(lambda x: AverageTrueRange(x['High'], x['Low'], x['Close'], window=14).average_true_range()).reset_index(level=0, drop=True)

    # Volume
    df['OBV'] = df.groupby('Ticker').apply(lambda x: OnBalanceVolume(x['Close'], x['Volume']).calculate()).reset_index(level=0, drop=True)

    # Oscillator
    df['RSI_14'] = df.groupby('Ticker')['Close'].transform(lambda x: RSIIndicator(close=x, window=14).rsi())

    # Risk-free rate
    risk_free_rate = 0.02 / 252

    df['Sharpe_Ratio'] = df.groupby('Ticker')['returns'].transform(lambda x: x.rolling(21).apply(lambda r: (r.mean() - risk_free_rate) / r.std() if r.std() else np.nan))
    df['Sortino_Ratio'] = df.groupby('Ticker')['returns'].transform(lambda x: x.rolling(21).apply(lambda r: (r.mean() - risk_free_rate) / r[r < 0].std() if r[r < 0].std() else np.nan))

    cumulative = df.groupby('Ticker')['returns'].transform(lambda x: (1 + x).cumprod())
    peak = cumulative.groupby(df['Ticker']).cummax()
    drawdown = (cumulative - peak) / peak
    df['Max_Drawdown'] = drawdown.groupby(df['Ticker']).transform(lambda x: x.rolling(21).min())

    df['Calmar_Ratio'] = df['returns'].groupby(df['Ticker']).transform(lambda x: x.rolling(21).mean()) / df['Max_Drawdown'].abs()
    df['Volatility'] = df.groupby('Ticker')['returns'].transform(lambda x: x.rolling(21).std() * np.sqrt(252))
    df['Cumulative_Return'] = df.groupby('Ticker')['returns'].transform(lambda x: (1 + x).cumprod() - 1)

    df['Ticker_returns'] = df.groupby('Ticker')['Close'].pct_change()
    df['Covariance'] = df.groupby('Ticker').apply(lambda x: x['returns'].rolling(21).cov(x['Ticker_returns'])).reset_index(level=0, drop=True)
    df['Beta'] = df.groupby('Ticker').apply(lambda x: x['Covariance'] / x['Ticker_returns'].rolling(21).var()).reset_index(level=0, drop=True)
    df['Alpha'] = df.groupby('Ticker').apply(lambda x: (x['returns'].rolling(21).mean() - risk_free_rate) - x['Beta'] * (x['Ticker_returns'].rolling(21).mean() - risk_free_rate)).reset_index(level=0, drop=True)

    df['CAGR'] = df.groupby('Ticker')['Close'].transform(lambda x: (x / x.shift(252)) ** (1 / 1) - 1 if len(x) >= 252 else np.nan)
    df['Total_Return'] = df.groupby('Ticker')['Close'].transform(lambda x: (x - x.shift(252)) / x.shift(252))

    df['VaR_95'] = df.groupby('Ticker')['returns'].transform(lambda x: x.rolling(252).quantile(0.05))
    df['CVaR_95'] = df.groupby('Ticker')['returns'].transform(lambda x: x.rolling(252).apply(lambda r: r[r <= r.quantile(0.05)].mean() if not r[r <= r.quantile(0.05)].empty else np.nan))
    df['Drawdown_Duration'] = df['Max_Drawdown'].lt(0).groupby(df['Ticker']).transform(lambda x: x.rolling(252).sum())

    adx = df.groupby('Ticker').apply(lambda x: pd.DataFrame({
        'ADX': ADXIndicator(x['High'], x['Low'], x['Close'], window=14).adx(),
        'DMI_plus': ADXIndicator(x['High'], x['Low'], x['Close'], window=14).adx_pos(),
        'DMI_minus': ADXIndicator(x['High'], x['Low'], x['Close'], window=14).adx_neg()
    }, index=x.index)).reset_index(level=0, drop=True)
    df = df.join(adx)

    # Ensure rolling window size is valid for each group
    df['avg_daily_volume'] = df.groupby('Ticker')['Volume'].transform(
    lambda x: x.rolling(252).mean() if len(x) >= 252 else np.nan
)
    df['correlation_with_benchmark'] = df.groupby('Ticker').apply(lambda x: x['returns'].rolling(252).corr(x['Ticker_returns'])).reset_index(level=0, drop=True)

    for period in [10, 30, 50, 100, 200]:
        df[f"SMA_{period}"] = df.groupby('Ticker')['Close'].transform(lambda x: SMAIndicator(close=x, window=period).sma_indicator())
        df[f"EMA_{period}"] = df.groupby('Ticker')['Close'].transform(lambda x: EMAIndicator(close=x, window=period).ema_indicator())

    df['Skewness'] = df.groupby('Ticker')['returns'].transform(lambda x: x.rolling(252).apply(pd.Series.skew))
    df['Kurtosis'] = df.groupby('Ticker')['returns'].transform(lambda x: x.rolling(252).apply(pd.Series.kurt))

    df['Win_Rate'] = df.groupby('Ticker')['returns'].transform(lambda x: (x > 0).rolling(252).mean())
    df['Avg_Gain'] = df.groupby('Ticker')['returns'].transform(lambda x: x.where(x > 0).rolling(252).mean())
    df['Avg_Loss'] = df.groupby('Ticker')['returns'].transform(lambda x: x.where(x < 0).rolling(252).mean())

    df['Up_Capture_Ratio'] = df.groupby('Ticker').apply(lambda x: x['returns'].rolling(252).mean() / x['Ticker_returns'].rolling(252).mean()).reset_index(level=0, drop=True)
    df['Down_Capture_Ratio'] = df.groupby('Ticker').apply(lambda x: x['returns'].where(x['returns'] < 0).rolling(252).mean() / x['Ticker_returns'].where(x['Ticker_returns'] < 0).rolling(252).mean()).reset_index(level=0, drop=True)

    # Ulcer Index
    def ulcer_index(series):
        max_close = series.cummax()
        drawdown = (series - max_close) / max_close
        return np.sqrt((drawdown ** 2).mean())

    df['Ulcer_Index'] = df.groupby('Ticker')['Close'].transform(lambda x: x.rolling(14).apply(ulcer_index))

    # Rate of Change (ROC)
    df['ROC_14'] = df.groupby('Ticker')['Close'].transform(lambda x: x.pct_change(periods=14))
    
    return df


if __name__ == "__main__":
    file_path = "/Users/imperator/Documents/STUDIES/UNIVERSITY OF GHANA/RESEARCH WORK/CORCHIL KELLY KWAME/PORTFOLIO OPTIMIZATION/PROJECT CODE/Dimensionality-Reduction-PortfolioOptimization/ETFs_datasets/Final_csv/merged_etfs.csv"
    output_path = "/Users/imperator/Documents/STUDIES/UNIVERSITY OF GHANA/RESEARCH WORK/CORCHIL KELLY KWAME/PORTFOLIO OPTIMIZATION/PROJECT CODE/Dimensionality-Reduction-PortfolioOptimization/ETFs_datasets/Final_csv/indicators_output_new.csv"

    try:
        indicators_df = calculate_indicators_with_moving_averages(file_path)
        indicators_df.to_csv(output_path, index=False)
        print("Indicators calculated and saved to:", output_path)
    except Exception as e:
        print(f"Error: {e}")

Error: index 13 is out of bounds for axis 0 with size 11


/var/folders/4r/xd1m46rs2r1gx3mmpw3z84pw0000gn/T/ipykernel_51738/331979028.py:32: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  bb = df.groupby('Ticker', group_keys=False).apply(
/var/folders/4r/xd1m46rs2r1gx3mmpw3z84pw0000gn/T/ipykernel_51738/331979028.py:38: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  macd = df.groupby('Ticker', group_keys=False).apply(


### Splitting the original dataset

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os
import numpy as np

# Load your dataset
file_path = ('/Users/imperator/Documents/STUDIES/UNIVERSITY OF GHANA/RESEARCH WORK/CORCHIL KELLY KWAME/PORTFOLIO OPTIMIZATION/PROJECT CODE/Dimensionality-Reduction-PortfolioOptimization/EDA/Notebooks/datacollection_preprocessing/cleaned_financial_data_final.csv') # Replace with your actual file path
data = pd.read_csv(file_path)

# Strip whitespace from column names
data.columns = data.columns.str.strip()

# Inspect unique values in the 'Date' column to identify potential formatting issues
print("Unique values in Date column:")
print(data['Date'].unique())

# Function to attempt parsing with different date formats
def parse_date(date_string):
    formats = ["%Y-%m-%d", "%m/%d/%Y", "%d/%m/%Y", "%Y%m%d"]  # Add more formats if needed
    for fmt in formats:
        try:
            return pd.to_datetime(date_string, format=fmt)
        except ValueError:
            continue  # Try the next format if this one fails
    return pd.NaT  # Return NaT if none of the formats work

# Apply the parsing function to the 'Date' column
data['Date'] = data['Date'].astype(str).str.strip()  # Ensure it's string and strip whitespace
data['Date'] = data['Date'].apply(parse_date)

# Drop rows where 'Date' could not be parsed
data = data.dropna(subset=['Date'])

# Check if 'Date' column is successfully converted to datetime objects
if pd.api.types.is_datetime64_any_dtype(data['Date']):
    print("'Date' column successfully converted to datetime objects.")
else:
    print("Failed to convert 'Date' column to datetime objects.")


# Define features (X) and target (y)
# Adjust column names based on your dataset
features = data[[
"Date", "Ticker", "Close", "High", "Low", "Open", "Volume", "returns", 
    "SMA_10", "EMA_10", "SMA_20", "EMA_20", "SMA_30", "EMA_30", "SMA_50", "EMA_50", 
    "SMA_100", "EMA_100", "SMA_200", "EMA_200", 
    "BB_Upper_20", "BB_Middle_20", "BB_Lower_20", 
    "BB_Upper_50", "BB_Middle_50", "BB_Lower_50", 
    "RSI_14", "RSI_30", "MACD", "MACD_Signal", 
    "ADX_14", "DMI_plus_14", "DMI_minus_14", 
    "ADX_30", "DMI_plus_30", "DMI_minus_30", 
    "ATR_14", "ATR_30", "Stoch_K", "Stoch_D", "OBV", 
    "Beta", "Alpha", "Avg_Daily_Volume", "Skewness", "Kurtosis", "Win_Rate", 
    "Up_Capture_Ratio", "cpi", "core_cpi", "pce_price_index", "core_pce_price_index", 
    "import_price_index", "export_price_index", "real_gdp", "industrial_production", 
    "capacity_utilization", "durable_goods_orders", "real_pce", "real_private_investment", 
    "net_exports", "govt_expenditures", "unemployment_rate", "labor_force_participation", 
    "jobless_claims", "nonfarm_payrolls", "average_hourly_earnings", "job_openings", 
    "quits_rate", "labor_productivity", "fed_funds_rate", "treasury_10y", 
    "treasury_3m", "baa_yield", "aaa_yield", "consumer_credit", "money_supply_m2", 
    "mortgage_rate_30y", "bank_prime_rate", "credit_card_rate", 
    "leading_economic_index", "weekly_economic_index", "housing_starts", 
    "building_permits", "new_home_sales", "home_price_index", 
    "rental_vacancy_rate", "umich_consumer_sentiment"
]]
target = data['returns']

# Handle missing values by filling with 0 or dropping
features = features.fillna(0)  # Or features.dropna()
target = target.fillna(0)  # Or target.dropna()

# # Scale the features
# scaler = StandardScaler()
# features_scaled = scaler.fit_transform(features)

# Split the dataset into training and testing sets
# the train_size is 70% and the test_size is 30%
X_train, X_test, y_train, y_test, metadata_train, metadata_test = train_test_split(
    features, target, test_size=0.3, random_state=42)

# Retain 'ticker' and 'Date' columns for reference
metadata = data[['Ticker', 'Date']]

# Split the metadata into training and testing sets (same split as features and target)
# metadata_train, metadata_test = train_test_split(
#     metadata, test_size=0.30, random_state=42
# )

metadata = data.loc[features.index, ['Ticker', 'Date']]
metadata_train, metadata_test = train_test_split(
    metadata, test_size=0.3, random_state=42
)



# # Convert scaled features back to DataFrame with original column names
# X_train = pd.DataFrame(X_train, columns=features.columns)
# X_test = pd.DataFrame(X_test, columns=features.columns)

base_dir = "/Users/imperator/Documents/STUDIES/UNIVERSITY OF GHANA/RESEARCH WORK/CORCHIL KELLY KWAME/PORTFOLIO OPTIMIZATION/PROJECT CODE/Dimensionality-Reduction-PortfolioOptimization/EDA/ETFs_datasets/Model Data/Pre-Dim Reduction"
train_dir = os.path.join(base_dir, "Training_Dataset_1")
test_dir = os.path.join(base_dir, "Test_Dataset_1")

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Save features and targets
X_train.to_csv(os.path.join(train_dir, "X_train.csv"), index=False)
y_train.to_csv(os.path.join(train_dir, "y_train.csv"), index=False)
metadata_train.to_csv(os.path.join(train_dir, "metadata_train.csv"), index=False)

X_test.to_csv(os.path.join(test_dir, "X_test.csv"), index=False)
y_test.to_csv(os.path.join(test_dir, "y_test.csv"), index=False)
metadata_test.to_csv(os.path.join(test_dir, "metadata_test.csv"), index=False)

print("✅ All files saved successfully.")

# # Save metadata to CSV files
# metadata_train_file = os.path.join("/Users/imperator/Documents/STUDIES/UNIVERSITY OF GHANA/RESEARCH WORK/CORCHIL KELLY KWAME/PORTFOLIO OPTIMIZATION/PROJECT CODE/Dimensionality-Reduction-PortfolioOptimization/EDA/ETFs_datasets/Model Data/Pre-Dim Reduction/Training_Dataset")
# metadata_test_file = os.path.join("/Users/imperator/Documents/STUDIES/UNIVERSITY OF GHANA/RESEARCH WORK/CORCHIL KELLY KWAME/PORTFOLIO OPTIMIZATION/PROJECT CODE/Dimensionality-Reduction-PortfolioOptimization/EDA/ETFs_datasets/Model Data/Pre-Dim Reduction/Test_Dataset")

# metadata_train.to_csv(metadata_train_file, index=False)
# metadata_test.to_csv(metadata_test_file, index=False)

# # Save training and testing sets to CSV files in the specified folder
# train_features_file = os.path.join("/Users/imperator/Documents/STUDIES/UNIVERSITY OF GHANA/RESEARCH WORK/CORCHIL KELLY KWAME/PORTFOLIO OPTIMIZATION/PROJECT CODE/Dimensionality-Reduction-PortfolioOptimization/ETFs/Model Data/EDA/ETFs_datasets/Model Data/Pre-Dim Reduction/Test_Dataset/X_train1.csv")
# test_features_file = os.path.join("/Users/imperator/Documents/STUDIES/UNIVERSITY OF GHANA/RESEARCH WORK/CORCHIL KELLY KWAME/PORTFOLIO OPTIMIZATION/PROJECT CODE/Dimensionality-Reduction-PortfolioOptimization/ETFs/Model Data/EDA/ETFs_datasets/Model Data/Pre-Dim Reduction/Training_Dataset/X_test1.csv")
# train_target_file = os.path.join("/Users/imperator/Documents/STUDIES/UNIVERSITY OF GHANA/RESEARCH WORK/CORCHIL KELLY KWAME/PORTFOLIO OPTIMIZATION/PROJECT CODE/Dimensionality-Reduction-PortfolioOptimization/ETFs/Model Data/EDA/ETFs_datasets/Model Data/Pre-Dim Reduction/Training_Dataset/y_train1.csv")
# test_target_file = os.path.join("/Users/imperator/Documents/STUDIES/UNIVERSITY OF GHANA/RESEARCH WORK/CORCHIL KELLY KWAME/PORTFOLIO OPTIMIZATION/PROJECT CODE/Dimensionality-Reduction-PortfolioOptimization/ETFs/Model Data/EDA/ETFs_datasets/Model Data/Pre-Dim Reduction/Test_Dataset/y_test1.csv")

# X_train.to_csv(train_features_file, index=False)
# X_test.to_csv(test_features_file, index=False)
# y_train.to_csv(train_target_file, index=False)
# y_test.to_csv(test_target_file, index=False)

# # Print confirmation messages
# print(f"Training features saved to {train_features_file}")
# print(f"Testing features saved to {test_features_file}")
# print(f"Training target saved to {train_target_file}")
# print(f"Testing target saved to {test_target_file}")
# print(f"Metadata for training saved to {metadata_train_file}")
# print(f"Metadata for testing saved to {metadata_test_file}")


Unique values in Date column:
['11/04/2005' '12/04/2005' '13/04/2005' ... '07/04/2025' '08/04/2025'
 '09/04/2025']
'Date' column successfully converted to datetime objects.


ValueError: not enough values to unpack (expected 6, got 4)

### testing the MLP

In [18]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import os
import time
import tracemalloc
import psutil
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# ---------- CONFIGURATION ----------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

base_dir = '/Users/imperator/Documents/STUDIES/UNIVERSITY OF GHANA/RESEARCH WORK/CORCHIL KELLY KWAME/PORTFOLIO OPTIMIZATION/PROJECT CODE/Dimensionality-Reduction-PortfolioOptimization/EDA'
directory = os.path.join(base_dir, "ETFs_datasets/Model Data/Pre-Dim Reduction")
train_data_path = os.path.join(directory, "Training_Dataset")
test_data_path = os.path.join(directory, "Test_Dataset")
# ------------------------------------

# Step 1: Load data (preserve all columns)
def load_csv(path):
    return pd.read_csv(path)

X_train = load_csv(f"{train_data_path}/X_train_new.csv")
y_train = load_csv(f"{train_data_path}/y_train_new.csv")
X_test = load_csv(f"{test_data_path}/X_test_new.csv")
y_test = load_csv(f"{test_data_path}/y_test_new.csv")

# Step 2: Separate Date column if present
date_col = None
for col in X_train.columns:
    if 'date' in col.lower():
        date_col = col
        break

if date_col:
    X_train_dates = X_train[date_col]
    X_test_dates = X_test[date_col]
    # Drop date column for modeling
    X_train = X_train.drop(columns=[date_col])
    X_test = X_test.drop(columns=[date_col])
else:
    X_train_dates = None
    X_test_dates = None

# Step 3: Ensure numeric (object to numeric, handle missing values)
X_train = X_train.apply(pd.to_numeric, errors='coerce')
X_test = X_test.apply(pd.to_numeric, errors='coerce')
y_train = y_train.apply(pd.to_numeric, errors='coerce')
y_test = y_test.apply(pd.to_numeric, errors='coerce')

# Fill or drop NaNs as appropriate (here, fill with 0)
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)
y_train = y_train.fillna(0)
y_test = y_test.fillna(0)

# Step 4: Create validation split from training data
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, shuffle=False
)

# Step 5: Convert to PyTorch tensors and DataLoader
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).to(device)
X_val_tensor = torch.tensor(X_val.values, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).to(device)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# Step 6: Define MLP model
class MLP(nn.Module):
    def __init__(self, input_dim):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.model(x)

input_dim = X_train_tensor.shape[1]
model = MLP(input_dim).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Step 7: Train model with early stopping and track metrics
epochs = 100
patience = 10
best_val_loss = float('inf')
counter = 0
best_model_state = model.state_dict()

tracemalloc.start()
train_start_time = time.time()
cpu_start = psutil.cpu_percent(interval=None)
memory_start = psutil.virtual_memory().used

for epoch in range(epochs):
    model.train()
    train_losses = []
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        preds = model(X_batch)
        loss = criterion(preds, y_batch)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())

    model.eval()
    val_losses = []
    with torch.no_grad():
        for X_val_batch, y_val_batch in val_loader:
            val_preds = model(X_val_batch)
            val_loss = criterion(val_preds, y_val_batch)
            val_losses.append(val_loss.item())
    avg_train_loss = np.mean(train_losses)
    avg_val_loss = np.mean(val_losses)
    print(f"Epoch {epoch+1} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        counter = 0
        best_model_state = model.state_dict()
    else:
        counter += 1
        if counter >= patience:
            print("⏹️ Early stopping triggered.")
            break

# Step 8: Metrics after training
train_end_time = time.time()
current_mem, peak_mem = tracemalloc.get_traced_memory()
cpu_end = psutil.cpu_percent(interval=None)
memory_end = psutil.virtual_memory().used
tracemalloc.stop()

training_time = train_end_time - train_start_time
memory_usage_mb = peak_mem / (1024 ** 2)
cpu_usage = cpu_end - cpu_start

print(f"\n🕒 Training Time: {training_time:.2f} seconds")
print(f"💾 Peak Memory Usage: {memory_usage_mb:.2f} MB")
print(f"🧠 CPU Usage Delta: {cpu_usage:.2f}%")

# Step 9: Save best model
model.load_state_dict(best_model_state)
output_model_path = os.path.join(base_dir,"Notebooks/deep_learning_models/Without_DR/MLP1.pth")
os.makedirs(os.path.dirname(output_model_path), exist_ok=True)
torch.save(model.state_dict(), output_model_path)
print(f"✅ Best model saved to: {output_model_path}")

# Step 10: Generate predictions and save to CSV (with length alignment)
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor).cpu().numpy().flatten()
    y_true = y_test_tensor.cpu().numpy().flatten()

# Align columns before saving to CSV
if X_test_dates is not None:
    X_test_dates_aligned = pd.Series(X_test_dates).reset_index(drop=True)
    y_true_aligned = pd.Series(y_true).reset_index(drop=True)
    y_pred_aligned = pd.Series(y_pred).reset_index(drop=True)
    min_len = min(len(X_test_dates_aligned), len(y_true_aligned), len(y_pred_aligned))
    results = pd.DataFrame({
        'Date': X_test_dates_aligned[:min_len],
        'Actual': y_true_aligned[:min_len],
        'Predicted': y_pred_aligned[:min_len]
    })
else:
    results = pd.DataFrame({'Actual': y_true, 'Predicted': y_pred})

results.to_csv('mlp_predictions.csv', index=False)
print("✅ Predictions saved to mlp_predictions1.csv")

# Step 11: Evaluate predictive performance
mse = mean_squared_error(results['Actual'], results['Predicted'])
mae = mean_absolute_error(results['Actual'], results['Predicted'])
r2 = r2_score(results['Actual'], results['Predicted'])
print(f"MSE: {mse:.4f}, MAE: {mae:.4f}, R²: {r2:.4f}")


✅ Using device: cpu


/Users/imperator/Documents/STUDIES/UNIVERSITY OF GHANA/RESEARCH WORK/CORCHIL KELLY KWAME/PORTFOLIO OPTIMIZATION/PROJECT CODE/Dimensionality-Reduction-PortfolioOptimization/.venv/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([32, 2])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/imperator/Documents/STUDIES/UNIVERSITY OF GHANA/RESEARCH WORK/CORCHIL KELLY KWAME/PORTFOLIO OPTIMIZATION/PROJECT CODE/Dimensionality-Reduction-PortfolioOptimization/.venv/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([16, 2])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, targ

Epoch 1 | Train Loss: 389072850068408.7500 | Val Loss: 271315050.9959
Epoch 2 | Train Loss: 1138782121050.0867 | Val Loss: 0.0022
Epoch 3 | Train Loss: 328302712268.2966 | Val Loss: 0.0022
Epoch 4 | Train Loss: 183141171822.6465 | Val Loss: 0.0023
Epoch 5 | Train Loss: 112242877664.9248 | Val Loss: 0.0025
Epoch 6 | Train Loss: 110321585718.8709 | Val Loss: 0.0029
Epoch 7 | Train Loss: 69237073612.4652 | Val Loss: 0.0022
Epoch 8 | Train Loss: 22740954108.6334 | Val Loss: 0.0017
Epoch 9 | Train Loss: 10663651186.1988 | Val Loss: 0.0026
Epoch 10 | Train Loss: 25118837314.4111 | Val Loss: 0.0022
Epoch 11 | Train Loss: 11362162884.9065 | Val Loss: 0.0007
Epoch 12 | Train Loss: 16913912229.8732 | Val Loss: 0.0002
Epoch 13 | Train Loss: 1708864543.2861 | Val Loss: 0.0004
Epoch 14 | Train Loss: 26179648.6781 | Val Loss: 0.0004
Epoch 15 | Train Loss: 151830491.4661 | Val Loss: 0.0006
Epoch 16 | Train Loss: 100531828.6248 | Val Loss: 0.0020
Epoch 17 | Train Loss: 2295195040.6416 | Val Loss: 0.00